Here's a lesson on creating Multi-Agent Teams with AutoGen:

# Creating Multi-Agent Teams with AutoGen

## Setup

In [29]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [30]:
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("Async environment configured for Jupyter.")
except ImportError:
    print("Please install nest_asyncio with `pip install nest_asyncio`")

Async environment configured for Jupyter.


## Introduction

Multi-agent teams in AutoGen are groups of AI agents that collaborate to accomplish complex tasks. Teams provide structure for agent interactions and help coordinate their efforts effectively.

## Basic Team Creation

Let's create a simple team with two agents - a primary agent and a critic:

Basic `RoundRobinChat`:

"A team that runs a group chat with participants taking turns in a round-robin fashion to publish a message to all.

If a single participant is in the team, the participant will be the only speaker."

In [31]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"
)

# Create agents
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a creative writer who writes engaging content."
)

critic_agent = AssistantAgent(
    "critic", 
    model_client=model_client,
    system_message="You provide constructive feedback on writing. Respond with 'APPROVE' when satisfied."
)

# Create termination condition
termination = TextMentionTermination("APPROVE")

# Create team
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=termination
)

## Running the Team

There are two main ways to run a team:

### 1. Basic Run

In [32]:
# Run team synchronously
result = await team.run(task="Write a short story about a magical forest.")
result

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short story about a magical forest.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=31, completion_tokens=879), content="In a faraway land, nestled between the serrated peaks of the Drakenshire mountains, lay a forest shrouded in mist and mystery. This was Eldergrove, a magical realm where time twisted like the gnarled roots of the ancient oaks that dominated its landscape. Legends whispered among village folk spoke of trees that could grant wishes, streams that sang lullabies, and creatures that glimmered like stars under the moonlit sky.\n\nOne crisp morning, as dawn broke and painted the sky in strokes of amber and rose, a curious girl named Elara ventured into Eldergrove. Unbeknownst to her, this was no ordinary day; it was the Harvest Moon Festival, a time when the forest’s magic pulsed strongest and the air thrummed with enchantment.\n\nElara had heard 

In [33]:
result.messages

[TextMessage(source='user', models_usage=None, content='Write a short story about a magical forest.', type='TextMessage'),
 TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=31, completion_tokens=879), content="In a faraway land, nestled between the serrated peaks of the Drakenshire mountains, lay a forest shrouded in mist and mystery. This was Eldergrove, a magical realm where time twisted like the gnarled roots of the ancient oaks that dominated its landscape. Legends whispered among village folk spoke of trees that could grant wishes, streams that sang lullabies, and creatures that glimmered like stars under the moonlit sky.\n\nOne crisp morning, as dawn broke and painted the sky in strokes of amber and rose, a curious girl named Elara ventured into Eldergrove. Unbeknownst to her, this was no ordinary day; it was the Harvest Moon Festival, a time when the forest’s magic pulsed strongest and the air thrummed with enchantment.\n\nElara had heard tales of the forest

### 2. Streaming Run

In [34]:
from autogen_agentchat.base import TaskResult

# Stream messages as they're generated
async for message in team.run_stream(task="Write a short story about a magical forest."):
    if isinstance(message, TaskResult):
        print(f"Task completed: {message.stop_reason}")
    else:
        print(f"{message.source}: {message.content}")

user: Write a short story about a magical forest.
primary: In a realm that existed just beyond the rim of reality, there lay the Whispering Woods. This forest was cloaked in an aura of enchantment, with trees towering towards the heavens, their trunks spiraled like ancient staircases, and their leaves shimmering with iridescent colors that changed with the time of day. At dawn, they glowed an ethereal gold; at midday, they burned a brilliant emerald; and at twilight, they blushed with shades of rose and lavender.

Among those who had heard of the Whispering Woods was a young boy named Finn. He had always been entranced by the stories told by the elders of his village—a place where wishes could bloom like flowers and where the air sparkled with possibility. Yet, these tales came with a warning: the woods were not merely a place of beauty; they were alive, and they could be fickle.

One fateful afternoon, driven by an insatiable curiosity, Finn decided to seek out the forest himself. As 

## Team Types

AutoGen provides several team types for different use cases:

### 1. RoundRobinGroupChat
Agents take turns in a fixed order. Good for structured discussions.

In [5]:
agent1 = AssistantAgent(
    "Writer",
    model_client=model_client,
    system_message="You are a helpful assistant."
)

agent2 = AssistantAgent(
    "Editor",
    model_client=model_client,
    system_message="You provide constructive feedback on writing. Respond with 'APPROVE' when satisfied."
)

team = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=termination
)

output = await team.run(task="Write 3 jokes about the tragedy of life.")
output

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write 3 jokes about the tragedy of life.', type='TextMessage'), TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=123), content='Sure! Here are three jokes that touch on the tragedy of life with a humorous twist:\n\n1. Why did the nihilist refuse to play cards?\n   Because he couldn\'t deal with the hand life dealt him!\n\n2. Life is like a rollercoaster—full of ups and downs. \n   But nobody warned me it would leave me feeling queasy and screaming the whole way!\n\n3. They say life is a journey, but mine feels more like a GPS with no signal— \n   always making wrong turns and asking if I want to reroute to "disappointment"!\n\nHope you enjoyed these!', type='TextMessage'), TextMessage(source='Editor', models_usage=RequestUsage(prompt_tokens=167, completion_tokens=4), content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

In [18]:
from autogen_agentchat.conditions import MaxMessageTermination

termination = MaxMessageTermination(max_messages=3)

team = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=termination
)

output = await team.run(task="Write 3 jokes about the tragedy of life.")
output

/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_95302/4079043465.py:10: RuntimeWarning: coroutine 'BaseGroupChat.run' was never awaited
  output = await team.run(task="Write 3 jokes about the tragedy of life.")


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write 3 jokes about the tragedy of life.', type='TextMessage'), TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=653, completion_tokens=119), content='Sure, here are three fresh jokes about the tragedy of life:\n\n1. Why did life bring a ladder to the bar?\n   Because it wanted to help everyone reach new heights... only to remind them it\'s all downhill from there!\n\n2. I asked my friend how he deals with the tragedies of life.\n   He said, "I just pretend it’s all a plot twist in a sitcom. I keep waiting for the laugh track!"\n\n3. Life is like a game of chess: full of strategies and sacrifices, but at the end of the day, everyone just seems to lose their queen!', type='TextMessage'), TextMessage(source='Editor', models_usage=RequestUsage(prompt_tokens=903, completion_tokens=207), content='The new jokes you crafted have some clever ideas, but they could use a bit more punch to truly shine.

In [19]:
output.messages

[TextMessage(source='user', models_usage=None, content='Write 3 jokes about the tragedy of life.', type='TextMessage'),
 TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=653, completion_tokens=119), content='Sure, here are three fresh jokes about the tragedy of life:\n\n1. Why did life bring a ladder to the bar?\n   Because it wanted to help everyone reach new heights... only to remind them it\'s all downhill from there!\n\n2. I asked my friend how he deals with the tragedies of life.\n   He said, "I just pretend it’s all a plot twist in a sitcom. I keep waiting for the laugh track!"\n\n3. Life is like a game of chess: full of strategies and sacrifices, but at the end of the day, everyone just seems to lose their queen!', type='TextMessage'),
 TextMessage(source='Editor', models_usage=RequestUsage(prompt_tokens=903, completion_tokens=207), content='The new jokes you crafted have some clever ideas, but they could use a bit more punch to truly shine. Here’s my feedbac

In [21]:
# Create a new team with an external termination condition.
from autogen_agentchat.conditions import ExternalTermination
from autogen_agentchat.ui import Console
import asyncio

text_termination = TextMentionTermination("APPROVE")
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=2037, completion_tokens=162), content="In whispered hues of amber light,  \nThe trees don cloaks of gold and rust,  \nAs breezes dance in soft delight,  \nAnd leaves fall gently, drifted dust.  \n\nCrisp air wraps the world, a tender sigh,  \nPumpkin patches glow in twilight's embrace,  \nClouds of mist drift low, like dreams that fly,  \nAs harvest moons illuminate the space.  \n\nNature's palette, a vibrant show,  \nWith chilly nights and warm fires' glow,  \nIn every rustle, the seasons weave,  \nA tapestry of memories, dreams conceived.  \n\nSo let us wander through this golden maze,  \nWhere time slows down in autumn’s gaze,  \nWith hearts aglow and spirits free,  \nIn fall's sweet magic, we find the key.  ", type='TextMessage')], stop_reason='External termination requested

# SelectorGroupChat 
[`SelectorGroupChat`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat) is a group chat that uses an AI model to intelligently choose the next speaker based on context.

 How it works:
 - When given a task via run() or run_stream():
   1. The model analyzes conversation history and agent profiles to pick the next speaker
   2. Selected agent provides a response that's shared with all participants 
   3. Process repeats until termination condition is met

 Key features:
 - By default, prevents same speaker twice in a row (override with allow_repeated_speaker=True)
 - Can use custom selection function instead of default model
 - Maintains conversation context between tasks
 - Call reset() to clear conversation history
 - Returns TaskResult with full conversation when complete

### 2. SelectorGroupChat 
Uses an AI model to dynamically choose the next speaker based on context.

In [26]:
from autogen_agentchat.teams import SelectorGroupChat

termination =TextMentionTermination("APPROVE")

agent1 = AssistantAgent(
    "Writer",
    model_client=model_client,
    system_message="You are a creative joke writer."
)

agent2 = AssistantAgent(
    "Editor",
    model_client=model_client,
    system_message="You take in the score from the Ranker and provide feedback on the jokes in case they are below 5, if not you return with 'APPROVE'."
)

agent3 = AssistantAgent(
    "Ranker",
    model_client=model_client,
    system_message="You rank the quality of the jokes, giving an overall score from 1 to 10."
)

team = SelectorGroupChat(
    participants=[agent1, agent2, agent3],
    model_client=model_client,
    termination_condition=termination
)

In [27]:
output = await team.run(task="Write 3 jokes about the tragedy of life.")

In [28]:
output.messages

[TextMessage(source='user', models_usage=None, content='Write 3 jokes about the tragedy of life.', type='TextMessage'),
 TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=150), content='Sure, here are three jokes that touch on the tragedy of life with a humorous twist:\n\n1. **Why did the existential philosopher bring a ladder to therapy?**  \n   Because he wanted to reach higher understanding, but all he got was stuck in the same old rungs of despair!\n\n2. **What did the pessimistic comedian say after his set?**  \n   “Life is like a bad punchline—everyone saw it coming, but we still have to laugh to keep from crying!”\n\n3. **Why did the soul bring a GPS to its midlife crisis?**  \n   Because it got lost in the tragic journey of life and figured it might need directions to happiness—turns out, every route was a detour!', type='TextMessage'),
 TextMessage(source='Editor', models_usage=RequestUsage(prompt_tokens=210, completion_tokens=81), cont

### 3. Swarm

Agents communicate through explicit handoffs, allowing targeted interactions.

See the [docs for more infor on this advanced multi-agent pattern.](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/swarm.html) 

## Best Practices

1. Choose the right team type for your use case:
   - RoundRobinGroupChat for structured, turn-based interactions
   - SelectorGroupChat for dynamic, context-aware speaker selection
   - Swarm for explicit handoffs between agents

2. Use appropriate termination conditions:
   - TextMentionTermination for keyword-based stopping
   - MaxMessageTermination for limiting conversation length
   - ExternalTermination for manual control

3. Reset teams between unrelated tasks to clear conversation history

4. Use streaming for real-time monitoring of team interactions

5. Start with simple teams and add complexity as needed

Remember that teams add overhead compared to single agents, so use them only when the task complexity justifies multi-agent collaboration.